In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

torch.manual_seed(199)



In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DTYPE = torch.float16

In [3]:
random_affine = torchvision.transforms.RandomAffine(degrees=20,
                                                    scale=(0.8, 1.2),
                                                    translate=(0.2, 0.2),
                                                    interpolation=torchvision.transforms.InterpolationMode.BICUBIC)

release = torchvision.transforms.Resize((16,16))

In [4]:
transform1 = torchvision.transforms.Compose([random_affine,
                                             torchvision.transforms.Resize((32,32)),
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.ConvertImageDtype(torch.half)
                                             ])

transform2 = torchvision.transforms.Compose([torchvision.transforms.Resize((32,32)),
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.ConvertImageDtype(torch.half)
                                             ])

In [5]:
BATCH_SIZE = 128

In [6]:
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=8, pin_memory=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=8, pin_memory=True)

In [7]:
#Defining the convolutional neural network
class myLeNet5(nn.Module):
    def __init__(self):
        super(myLeNet5, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=0),
            #nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=0),
            #nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(4 * 5 * 5, 50)
        self.relu = nn.ReLU()
        #self.fc1 = nn.Linear(120, 84)
        #self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        y = self.layer1(x)
        y = self.layer2(y)
        y = y.reshape(y.size(0), -1)
        y = self.fc(y)
        y = self.relu(y)
        #y = self.fc1(y)
        #y = self.relu1(y)
        y = self.fc2(y)
        return y

In [8]:
net = myLeNet5()
net = net.train().to(device=DEVICE,dtype=DTYPE)
optimizer = torch.optim.SGD(net.parameters(), lr=0.005, momentum=0.8)
loss_func = torch.nn.CrossEntropyLoss()

In [9]:
net = net.train().to(device=DEVICE)
for epoch in range(34):  # loop over the dataset multiple times

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs=inputs.to(device=DEVICE)
        labels=labels.to(device=DEVICE)
        

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))
    running_loss = 0.0

print('Finished Training')

[1] loss: 1079.535
[2] loss: 1064.225
[3] loss: 743.382
[4] loss: 444.136
[5] loss: 337.680
[6] loss: 289.270
[7] loss: 248.571
[8] loss: 224.032
[9] loss: 208.218
[10] loss: 195.624
[11] loss: 184.047
[12] loss: 177.562
[13] loss: 168.390
[14] loss: 161.094
[15] loss: 154.434
[16] loss: 151.778
[17] loss: 149.262
[18] loss: 146.310
[19] loss: 139.349
[20] loss: 137.123
[21] loss: 136.440
[22] loss: 131.911
[23] loss: 129.554
[24] loss: 129.618
[25] loss: 126.204
[26] loss: 123.387
[27] loss: 123.726
[28] loss: 121.930
[29] loss: 120.057
[30] loss: 119.089
[31] loss: 117.828
[32] loss: 117.085
[33] loss: 115.800
[34] loss: 113.674
Finished Training


In [10]:
mynet=net.eval()
all_counter=0
correct_counter=0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    out = mynet(inputs)
    out = out.detach().cpu().argmax(1)
    t = labels.cpu()
    for m in range(len(t)):
        all_counter += 1
        if t[m] == out[m]:
            correct_counter += 1

print(correct_counter, all_counter, correct_counter / all_counter)

9717 10000 0.9717


In [11]:
print(mynet.cpu())

myLeNet5(
  (layer1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(4, 4, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=100, out_features=50, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [12]:
#2  max pool        0.9682
#2  avg pool+relu   0.9707
#2  max pool+relu   0.9772
#20 max pool+relu   0.9902
#20 去全连结 且120->80   0.9906
#25 去全连结 且120->80   0.9913
# 97.12

In [13]:
print(mynet.state_dict())


OrderedDict([('layer1.0.weight', tensor([[[[-0.0239,  0.0255,  0.3816,  0.7280,  0.4724],
          [ 0.0314,  0.1293,  0.5293,  0.5298,  0.2379],
          [ 0.2742,  0.3469,  0.3896,  0.2325,  0.2634],
          [ 0.5444,  0.2056,  0.4319,  0.2451,  0.0983],
          [ 0.3674,  0.3203,  0.3130,  0.2313,  0.3857]]],


        [[[ 0.6250, -0.0650, -0.3447, -0.5923, -0.6318],
          [ 0.7290,  0.0724,  0.0302, -0.4834, -0.5015],
          [ 0.4395,  0.3982, -0.0845, -0.4949, -0.4092],
          [ 0.6030,  0.1025, -0.1530, -0.3379, -0.4519],
          [ 0.3616, -0.1006, -0.3518, -0.4556, -0.5449]]],


        [[[ 0.1493, -0.7427, -0.9502, -0.8442, -0.5151],
          [ 1.0547,  0.3035, -0.4792, -0.6924, -0.4734],
          [ 0.9829,  0.7739,  0.2517,  0.1840, -0.0998],
          [ 0.1542,  0.5405,  0.6567,  0.6938,  0.3000],
          [ 0.1903, -0.0292,  0.5068,  0.6562,  0.7891]]],


        [[[-0.2732, -0.2520, -0.2438,  0.0257, -0.0554],
          [-0.1650, -0.1920, -0.1687, -0.12

In [14]:
#torch.save(mynet,"mynet.pth")